# Table from web as Data Frame

## Import Packages
I am going to utilize BeautifulSoup, request as web table scraping tool

In [0]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import urllib.request
import re

## Web Scraping

In [0]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs(source, 'lxml')

## Using 'find, find_all' from BeautifulSoup to get rows and columns

In [0]:
# table = soup.table
table = soup.find('table')
table_rows = table.find_all('tr')

df_table = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text for i in td]
    df_table.append(row)

## list into Data Frame and merge into one column

In [0]:
# set column names
df = pd.DataFrame(df_table, columns=['1', '2', '3', '4', '5', '6', '7', '8', '9'])

# replace '\n' string into ''
df = df.replace('\n','', regex=True)

# merge all column values into one column
df_concat = pd.concat([df, df.unstack().reset_index(drop=True).rename('10')], axis=1)

## split value by character length, and ignoring 'Not assigned' value

In [0]:
# Splitting value by character length into 2 columns
df_concat['PostalCode'] = df_concat['10'].str[:3].replace('\n','', regex=True)
df_concat['BN'] = df_concat['10'].str[3:]

# Select 2 columns only for easy preprocessing
dfs = df_concat[['PostalCode', 'BN']]

# Ignoring specific string value
dfs = dfs[~(dfs['BN'] == "Not assigned")]

## split value by special character, and replace special character to others

In [0]:
# split value by spacieal character - first column
dfs['Borough'] = dfs['BN'].str.split('\(', expand=True)[0]

# split value by special character - second column, replace special character to other
dfs['Neighborhood'] = dfs['BN'].str.split('\(', expand=True)[1].replace(' /', ',', regex=True).replace('\)', '', regex=True)

# choose 3 columns as new Data Frame
dfs = dfs[['PostalCode', 'Borough', 'Neighborhood']]

## Result 

In [8]:
dfs.head(12)

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
10,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
dfs.shape

(103, 3)